# Calling kernel code from frontend code

First, we instantiate and start a server, as defined in the [tutorial for local notebooks](local-kernel.ipynb).

In [ ]:
import jchannel

In [ ]:
HOST = '127.0.0.1'
PORT = 8889

In [ ]:
server = await jchannel.start(host=HOST, port=PORT)

Then, we instantiate and open a channel, but with a different initialization function. The most important part is storing the [client representation of the Channel](https://hashiprobr.github.io/jupyter-jchannel-client/Channel.html) in order to retrieve it later.

The example below simply stores this representation in a global variable.

In [ ]:
channel = await server.open('''
    (channel) => {
        self.channel = channel;
    }
''')

Finally, we set the `handler` property of the channel to an object. The methods of this object define the API at the kernel.

The example below sets `handler` to an object with a single method called `divide`. This method simply divides the first argument by the second argument.

In [ ]:
class Handler:
    def divide(self, a, b):
        return a / b

channel.handler = Handler()

And that's it! You can now call any method of the API from the console.

The `call` method returns a `Promise` that can be awaited for the result.

Arguments and return values can have any types that are JSON-serializable.

``` js
await channel.call('divide', 1, 2);
```

```
0.5
```

If the Python method throws an exception, it is wrapped in a JavaScript exception.

``` js
await channel.call('divide', 1, 0);
```

```
Uncaught KernelError: ZeroDivisionError: division by zero
```